In [4]:
library(AnnotationDbi)
library(org.Hs.eg.db)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(hugene10sttranscriptcluster.db) 
library(hugene10stprobeset.db)
library(illuminaHumanv4.db)
library(readxl)
library(GEOquery)
library(stringr)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The f

## Далее единый код для всех исходных данных, необходимо не забывать менять название входного файла и выходного 

In [ ]:
file_path <- "data/raw_data/GSE16717_series_matrix.txt"
file_content <- readLines(file_path)

In [ ]:
file_content <- readLines(file_path)

table_data <- list()
start_index <- which(file_content == "!series_matrix_table_begin") + 1
end_index <- which(file_content == "!series_matrix_table_end") - 1

table_data <- list()

for (i in start_index:end_index) {
  parts <- strsplit(file_content[i], "\t")[[1]]
  if (i == start_index) {
    col_names <- parts
    col_names <- gsub('"', '', col_names)
  } else {
    table_data <- c(table_data, list(parts))
  }
}

In [ ]:
df <- as.data.frame(do.call(rbind, table_data))
colnames(df) <- col_names
head(df,2)

### Аннотирование полученных данных, нужно делать, потому что в исходных файлах вместо названий генов, кодировки с платформы

#### Код для GSE16717, GSE67220

In [ ]:
gene_ids <- unique(df$ID_REF)
gene_symbols <- mapIds(org.Hs.eg.db, keys = gene_ids, column = "SYMBOL", keytype = "ENTREZID")
df$ID_REF <- gene_symbols[df$ID_REF]
head(df$ID_REF, 10)

#### Код для GSE56033

In [ ]:
df <- as.data.frame(do.call(rbind, table_data))
colnames(df) <- col_names
multi_mapping <- which(lengths(symbol_list) > 1)
df <- df[-multi_mapping, ]
symbol_list <- select(hugene10sttranscriptcluster.db, keys = df$ID_REF, columns = c("SYMBOL"), keytype = "PROBEID", mult='first')
anyDuplicated(df$ID_REF)
anyDuplicated(symbol_list)
symbol_list_cleaned <- na.omit(symbol_list)
symbol_list_cleaned_unique <- symbol_list_cleaned[!duplicated(symbol_list_cleaned$PROBEID),]
merged_df <- merge(df, symbol_list_cleaned_unique, by.x = "ID_REF", by.y = "PROBEID", all.x = TRUE)
df$ID_REF <- ifelse(!is.na(merged_df$SYMBOL), merged_df$SYMBOL, df$ID_REF)

#### код для GSE56047, GSE65907

In [ ]:
# только для GSE65907 
df <- df[-1, ]

In [ ]:
df_copy <- df
df_copy$Gene <- mapIds(illuminaHumanv4.db, 
                        keys = df_copy$ID_REF,  
                        column = "SYMBOL",
                        keytype = "PROBEID")
head(df_copy)

In [ ]:
df_copy$ID_REF <- df_copy$Gene
df_copy <- subset(df_copy, select = -Gene)
head(df_copy)

Далее вместо звездочек надо указать номера GSE файла с которым проводилась работа

In [ ]:
write.csv(data, file = "data/raw_data/GSE****.csv", row.names = TRUE)

## Объединение всех df, которые были анатированы в данном протоколе, а также были аннотированы ранее

In [ ]:
#Перед выполнением следующей части для некоторый df стоит их почистить от лищних символов в названиях или значениях экспрессии
#например следующим образом 
#colnames(data) <- sub("\\.x$", "", colnames(data))

In [ ]:
df1 <- read.csv("data/raw_data/annotated_data.csv", header = TRUE)
df2 <- read.csv("data/raw_data/GSE16717.csv", header = TRUE)
df3 <- read.csv("data/raw_data/GSE56033.csv", header = TRUE)
df4 <- read.csv("data/raw_data/GSE65907.csv", header = TRUE)
df5 <- read.csv("data/raw_data/GSE67220.csv", header = TRUE)
df6 <- read.csv("data/raw_data/biohack/geneMatTable_GSE30483.csv", header = TRUE)
df7 <- read.csv("data/raw_data/biohack/geneMatTable_GSE47353.csv", header = TRUE)
df8 <- read.csv("data/raw_data/biohack/geneMatTable_GSE68759.csv", header = TRUE)
df9 <- read.csv("data/raw_data/biohack/geneMatTable_GSE75511.csv", header = TRUE)

df1$ID_REF <- as.character(df1$ID_REF)
df2$ID_REF <- as.character(df2$ID_REF)
df3$ID_REF <- as.character(df3$ID_REF)
df4$ID_REF <- as.character(df4$ID_REF)
df5$ID_REF <- as.character(df5$ID_REF)
df6$ID_REF <- as.character(df6$ID_REF)
df7$ID_REF <- as.character(df7$ID_REF)
df8$ID_REF <- as.character(df8$ID_REF)
df9$ID_REF <- as.character(df9$ID_REF)

common_values <- Reduce(intersect, list(df1$ID_REF, df2$ID_REF, df3$ID_REF, 
                                        df4$ID_REF, df5$ID_REF, df6$ID_REF, 
                                        df7$ID_REF, df8$ID_REF, df9$ID_REF))

filtered_df1 <- df1[df1$ID_REF %in% common_values, ]
filtered_df2 <- df2[df2$ID_REF %in% common_values, ]
filtered_df3 <- df3[df3$ID_REF %in% common_values, ]
filtered_df4 <- df4[df4$ID_REF %in% common_values, ]
filtered_df5 <- df5[df5$ID_REF %in% common_values, ]
filtered_df6 <- df6[df6$ID_REF %in% common_values, ]
filtered_df7 <- df7[df7$ID_REF %in% common_values, ]
filtered_df8 <- df8[df8$ID_REF %in% common_values, ]
filtered_df9 <- df9[df9$ID_REF %in% common_values, ]


all_filtered_data <- list(filtered_df1, filtered_df2, filtered_df3, 
                          filtered_df4, filtered_df5, filtered_df6, 
                          filtered_df7, filtered_df8, filtered_df9)

merged_data <- Reduce(function(x, y) merge(x, y, by = "ID_REF", all = TRUE), all_filtered_data)
merged_data <- merged_data[!duplicated(merged_data$ID_REF), ]
                      
merged_data_hackaton_IB_GB <- merged_data[!duplicated(merged_data$ID_REF), ]
write.csv(data, file = "data/for_analysis/merged_data_hackaton_IB_GB_correct.csv", row.names = TRUE)

## Получение meta даблица, содержащей следующие колонки GSE, GSM, age, sex 

### Можно использовать разные подходы для разных датасетов, т.к. не всегда подходит один метод

##### Для GSE16717

In [ ]:
library(GEOquery)

gse <- getGEO("GSE16717", GSEMatrix = TRUE)
metadata <- pData(gse[[1]])
ages <- as.numeric(sub("\\..*", "", metadata$age))
ages <- as.numeric(gsub("[^0-9.]", "", metadata$age))
ages <- as.integer(ages)
GSM <- rep(rownames(metadata))
GSE <- paste("GSE16717" )
ages <- as.numeric(ages)
sex <- rep(NA, length(ages))
df <- data.frame(GSE = GSE, GSM = GSM, age = ages, sex = sex)
head (df1, 3)
write.csv(df, file = "data/raw_data/GSE16717_annotation.csv", row.names = FALSE)

##### Для GSE67220

In [ ]:
file_path <- "data/raw_data/GSE67220_series_matrix.txt" 

file_content <- readLines(file_path)
ages <- c()

for (line in file_content) {
  if (startsWith(line, "!Sample_title")) {
    parts <- strsplit(line, "\t")[[1]]
    for (part in parts[-1]) {
      age <- as.numeric(gsub("[^0-9]", "", part))
      ages <- c(ages, age)
    }
  }
}
GSM_numbers <- c()
for (line in file_content) {
  if (startsWith(line, "!Sample_geo_accession")) {
    parts <- strsplit(line, "\t")[[1]]
    for (part in parts[-1]) {
      GSM_numbers <- c(GSM_numbers, part)
    }
  }
}
GSM <- rep(GSM_numbers)
GSE <- paste("GSE67220")
ages <- as.numeric(ages)
sex <- rep(NA, length(ages))

df <- data.frame(GSE = GSE, GSM = GSM, age = ages, sex = sex)
write.csv(df, file = "data/raw_data/GSE67220_annotation.csv", row.names = FALSE)

##### Для GSE56033

In [ ]:
file_path <- "data/raw_data/GSE56033_series_matrix.txt"
file_content <- readLines(file_path)
file_content <- paste(file_content, collapse = " ")

age_list <- regmatches(file_content, gregexpr("\\b(age \\(yrs\\)|age): \\d+\\b", file_content))[[1]]
age_list <- regmatches(file_content, gregexpr("\\b(\\d+): \\d+\\b", file_content))[[1]]
age_list <- c(age_list, age_list)

#age_list <- as.integer(gsub("\\D", "", age_list))
#print(length(age_list5))
gender_list <- regmatches(file_content, gregexpr("\\b(gender: (male|female))\\b", file_content))[[1]]
gender_list <- regmatches(file_content, gregexpr("\\b(Sex: (Male|Female))\\b", file_content))[[1]]

gender_list <- c(gender_list, gender_list)
gender_list <- gsub("(gender|Sex): ", "", gender_list)

gsm_list <- regmatches(file_content, gregexpr("\\b(GSM\\d+)\\b", file_content))[[1]]


GSE <- rep("GSE56033", length(gsm_list))
GSM <- rep(gsm_list)
sex <- rep(gender_list5)
ages <- as.numeric(age_list5)

GSM <- unique(GSM)
df <- data.frame(GSE = GSE, GSM = GSM, age = ages, sex = sex)

write.csv(df, file = "data/raw_data/GSE56033_annotation.csv", row.names = FALSE)

In [ ]:
###### GSE56047

In [ ]:
file_path <- "data/raw_data/GSE56047-GPL10558_series_matrix.txt"

GSM_numbers <- character(0) 
con <- file(file_path, "r") 
while (TRUE) {
  line <- readLines(con, n = 1) 
  if (length(line) == 0) break  
  if (startsWith(line, "!Sample_geo_accession")) {
    parts <- strsplit(line, "\t")[[1]]
    for (part in parts[-1]) {
      GSM_numbers <- c(GSM_numbers, gsub("\"", "", part))
    }
  }
}
close(con)

GSM_nmber <- unique(GSM_numbers)
length (GSM_nmber)
GSE <- rep("GSE56047", length(ages))
GSM <- rep(GSM_numbers)
sex <- rep(NA, length(ages))
ages <- as.numeric(unlist(ages))
df <- data.frame(GSE = GSE, GSM = GSM, age = ages, sex = sex)
head(dfGSE56047, 3)
dim(dfGSE56047)
write.csv(df, file = "data/raw_data/GSE56047_annotation.csv", row.names = FALSE)

### Собираем все вместе 

In [ ]:
data_GSE30483 <- read.csv("biohack/ages_df_GSE30483.csv", header = TRUE)
data_GSE47353 <- read.csv("biohack/ages_df_GSE47353.csv", header = TRUE)
data_GSE68759 <- read.csv("biohack/ages_df_GSE68759.csv", header = TRUE)
data_GSE75511 <- read.csv("biohack/ages_df_GSE75511.csv", header = TRUE)
data_GSE16717 <- read.csv("biohack/ages_df_GSE75511.csv", header = TRUE)
data <- read.csv("data/raw_data/annotated_data.csv", header = TRUE) 
data_GSE56047 <- read.csv("data/raw_data/GSE56047_annotation.csv". header = TRUE)
data_GSE56033 <- read.csv("data/raw_data/GSE56033_annotation.csv". header = TRUE)
data_GSE67220 <- read.csv("data/raw_data/GSE67220_annotation.csv". header = TRUE)
data_GSE16717 <- read.csv("data/raw_data/GSE16717_annotation.csv". header = TRUE)

Во всех данных необзодимо перевести названия всех столбцов в единый форма по следюущему приницу. Не забудьте поменять номе GSE 

In [ ]:
colnames(data_GSE30483)[colnames(data_GSE30483) == "gsms"] <- "GSM"
colnames(data_GSE30483)[colnames(data_GSE30483) == "ages"] <- "age"

In [ ]:
meta_data_correct <- rbind(data, data_GSE56047, data_GSE56033, data_GSE67220, data_GSE16717, data_GSE30483, data_GSE47353, data_GSE68759, data_GSE75511)
meta_data_correct <- lapply(meta_data_correct, function(x) gsub('^"|"$', '', x))

In [ ]:
write.csv(meta_data_correct, file = "data/for_analysis/merged_data_annotation_full.csv", row.names = FALSE)

Т.к. в GSE 56033 небыло разделение по полу, то это разделение совершалось отдельно в Rscript под названием *genes_annotation.r*
